In [1]:
import torch
import sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.utils import BoxUniform
from sbi.inference import prepare_for_sbi, simulate_for_sbi, SNPE, SNLE, SNRE
from sbi.analysis import pairplot
import numpy as np
from matplotlib import pyplot as plt
import pyccl as ccl
import pyccl.halos.profiles as prof
import pyccl.halos.concentration as conc

In [2]:
import torch
from torch import zeros, ones

In [3]:
cosmo = ccl.core.Cosmology(Omega_c=0.27 - 0.045, Omega_b=0.045, h=0.7, n_s=0.98, 
                     sigma8=0.8, Omega_k=0.0)

### Given the pre-made simulation data with noisy Diemer-Kravtsov, going to continue from Yuanyuan's tutorial here: https://github.com/deepskies/DeepLenSBI/blob/main/CLCosmo/sbi_test.ipynb


## Load simulation data

In [4]:
x_np = np.load('noisy_DK_NFW_profiles.npy') ##x='profiles'
np.shape(x_np)

(10000, 20)

In [5]:
logmassprior = utils.BoxUniform(low=13.7*torch.ones(1), high=14.7*torch.ones(1))
concmassprior = utils.BoxUniform(low=3*torch.ones(1), high=5*torch.ones(1))

In [6]:
prior = BoxUniform(-ones(2)*20, ones(2)*20)

In [7]:
theta_np = np.load('noisy_DK_massconc_pairs.npy')

In [26]:
np.shape(x_np)

(10000, 20)

In [25]:
np.shape(theta_np)

(10000, 2)

In [8]:
theta_np=theta_np.T

In [32]:
### for now, defining the "truth" as the first set of the simulated obs's / par's

theta_truth = theta_np[0]
x_truth = x_np[0]

In [9]:
theta = torch.as_tensor(theta_np, dtype=torch.float32)
x = torch.as_tensor(x_np, dtype=torch.float32)

In [18]:
### what is the truth here? it should just be one of the observations?
### along the line?

In [20]:
# the point of this project is to figure out if SBI is a valid way to estimate
# g cluster mass using weak lensing shear profiles -- that hasn't really come up
# with this simulation yet???

### start with defining the initial truths as just the first set of sol's? sim's?

In [ ]:
# SO the x --> PROFILES (CONVERGENCE) and then theta --> MASS+CONC
### profiles = simulateConvergenceProfile(samplemasses,noisysampleconcentrations)

## Train density estimator

In [13]:
# Create inference object: choose method and estimator
inferer = SNPE(prior, density_estimator="mdn", device="cpu")  # SNLE, SNRE

In [14]:
# Append training data
inferer = inferer.append_simulations(theta, x)

In [15]:
#Train
density_estimator = inferer.train(num_atoms=10, training_batch_size=50, 
                                  learning_rate=0.0005, 
                                  validation_fraction=0.1, 
                                  stop_after_epochs=20, max_num_epochs=1000, 
                                  clip_max_norm=5.0, calibration_kernel=None, 
                                  exclude_invalid_x=True, 
                                  resume_training=False, 
                                  discard_prior_samples=False, 
                                  use_combined_loss=False, 
                                  show_train_summary=False, 
                                  dataloader_kwargs=None)  
# Lots of training settings.


 Neural network successfully converged after 239 epochs.

## Obtain posterior

In [16]:
# Build posterior using trained density estimator
posterior = inferer.build_posterior(density_estimator)  # Posterior sampling settings.

In [33]:
# Generate samples
#theta_o = torch.as_tensor(theta_np[20], dtype=torch.float32)
#x_o = torch.as_tensor(x_np[20], dtype=torch.float32)
theta_o = torch.as_tensor(theta_truth, dtype=torch.float32) 
x_o = torch.as_tensor(x_truth, dtype=torch.float32)
samples = posterior.sample((10000,), x=x_o)

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

/global/homes/e/erinbari/.conda/envs/sbi_env/lib/python3.10/site-packages/pyknos/mdn/mdn.py:279: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2189.)
  zero_mean_samples, _ = torch.triangular_solve(


In [44]:
pip install pygtc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.3 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [41]:
#pairplot(samples[:, :5], points=theta_o, points_colors=["k"], upper="scatter", limits=[[0, 1]], figsize=(10,10));
#from matplotlib import pyplot as plt
#plt.show()



from chainconsumer import ChainConsumer
from matplotlib import pyplot as plt


np_samples=samples.numpy()
theta_truth=theta_o.numpy()
c = ChainConsumer()
print(np.shape(np_samples))
c.add_chain(np_samples, parameters=["Mass", "Concentration"])
#fig = c.plotter.plot(figsize="column", truth=theta_truth)


(10000, 2)


In [45]:
import pygtc

In [50]:
priors2d = ((-20,20),(20,20))

In [51]:
GTC = pygtc.plotGTC(chains=[np_samples], 
                     paramNames=['log mass','concentration'],
                     truths=theta_truth,
                     priors=priors2d,
                     figureSize='MNRAS_column')

GTC = pygtc.plotGTC(chains=[np_samples],
                    paramNames=['log mass', 'concentration'],
                    truths=theta_truth,
                    priors=priors2d,
                    figureSize='MNRAS_column',
                    do1dPlots=False, customTickFont={'family':'Arial', 'size':12})


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/TeX Live for SUSE Linux) (preloaded format=latex)
 restricted \write18 enabled.

kpathsea: Running mktexfmt latex.fmt
mktexfmt [ERROR]: -user mode but path setup is -sys type, bailing out.
I can't find the format file `latex.fmt'!




RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/TeX Live for SUSE Linux) (preloaded format=latex)
 restricted \write18 enabled.

kpathsea: Running mktexfmt latex.fmt
mktexfmt [ERROR]: -user mode but path setup is -sys type, bailing out.
I can't find the format file `latex.fmt'!




<Figure size 240x240 with 3 Axes>

In [42]:
fig = c.plotter.plot(figsize="column", truth=theta_truth)

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/TeX Live for SUSE Linux) (preloaded format=latex)
 restricted \write18 enabled.

kpathsea: Running mktexfmt latex.fmt
mktexfmt [ERROR]: -user mode but path setup is -sys type, bailing out.
I can't find the format file `latex.fmt'!




RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/TeX Live for SUSE Linux) (preloaded format=latex)
 restricted \write18 enabled.

kpathsea: Running mktexfmt latex.fmt
mktexfmt [ERROR]: -user mode but path setup is -sys type, bailing out.
I can't find the format file `latex.fmt'!




<Figure size 360x360 with 4 Axes>

In [37]:
fmtutil-sys --all

NameError: name 'fmtutil' is not defined

In [ ]:
#pygtc
